
## Project 2: Exploratory Data Analysis of US Flights Dataset
**Name:** Eleni Tesheshigo 

**Date:** February 2026  

### Project Objective
The goal of this project is to perform Exploratory Data Analysis (EDA) on the US Flights dataset to understand airfare behavior, passenger demand, market dominance, and trends over time using univariate, bivariate, and multivariate techniques.


## 1.  Data Loading, understanding  and Cleaning

This step involves loading and understanding the dataset, handling missing values, standardizing column names, converting data types, and removing duplicate records to ensure data quality before analysis.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

flights = pd.read_csv(
    r"C:\datanomics\python\Advanced python\panda\file\US Airline Flight Routes and Fares 1993-2024.csv",low_memory=False)

airline = flights.copy()
print(airline.columns)
print("Dataset Shape:", airline.shape)
airline['tbl'] = (
    airline['tbl']
    .str.lower()          
    .str.strip()         
    .str.replace(' ', '', regex=False))  
print(airline.duplicated().sum())
airline.info()
airline.describe()


###  Data Cleaning and Preparation

Data cleaning ensures accuracy and consistency. This step includes handling missing values, standardizing column names, converting data types, and removing unnecessary columns.


In [ ]:
# Fill missing market share values
airline['large_ms'] = airline['large_ms'].fillna(airline['large_ms'].median())
airline['lf_ms'] = airline['lf_ms'].fillna(airline['lf_ms'].median())


# Fill missing carrier names
airline['carrier_lg'] = airline['carrier_lg'].fillna('UNKNOWN')
airline['carrier_low'] = airline['carrier_low'].fillna('UNKNOWN')

# Drop irrelevant columns
airline.drop(columns=['Geocoded_City1', 'Geocoded_City2'], inplace=True)

# Fill missing fare values
airline['fare_lg'] = airline['fare_lg'].fillna(airline['fare'])
airline['fare_low'] = airline['fare_low'].fillna(airline['fare'])

# Convert year column datatype from int to datetime
airline['Year'] = pd.to_datetime(airline['Year'])

# Standardize column names
airline.columns = airline.columns.str.lower()

# Standardize text data
airline = airline.apply(
    lambda col: col.str.lower().str.strip() if col.dtype == "object" else col
)



### Why These Cleaning Methods?

- Missing values were filled instead of removed to avoid losing important observations.
- Market share values were filled with Median to represent full dominance when data was missing.
- Irrelevant geolocation columns were dropped because they do not influence pricing or demand.
- Standardization prevents inconsistencies and analysis errors.


## 2. Outlier Detection and Treatment

Outlier analysis is performed before distribution analysis to understand the presence of extreme values and decide whether they should be retained or removed.


### Why IQR Method?

The Interquartile Range (IQR) method is robust to skewed distributions and helps identify extreme values without assuming normality.


In [ ]:
Q1 = airline['fare'].quantile(0.25)
Q3 = airline['fare'].quantile(0.75)

IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = airline[
    (airline['fare'] < lower_bound) | (airline['fare'] > upper_bound)
]

print("IQR:", IQR)
print("Lower Bound:", lower_bound)
print("Upper Bound:", upper_bound)
print("Number of Outliers:", outliers.shape[0])


### interpretation 

any value that is below lower bound and above upper bound is outliers 

In [ ]:
# Select top 5 airlines by total passengers
top_5_airlines = (
    airline.groupby('carrier_lg')['passengers']
    .sum()
    .sort_values(ascending=False)
    .head(5)
    .index
)

# Filter dataset
top_airline_data = airline[airline['carrier_lg'].isin(top_5_airlines)]

# Violin plot
plt.figure(figsize=(12,6))
sns.violinplot(
    data=top_airline_data,
    x='carrier_lg',
    y='fare'
)

plt.title('Fare Distribution by Top 5 Airlines (Outlier Detection)')
plt.xlabel('Airline')
plt.ylabel('Fare (USD)')
plt.xticks(rotation=45)
plt.show()


### Outlier Decision and Justification

Outliers were retained because extreme fare values likely represent premium, long-distance, or low-competition routes. Removing them would distort real-world airline pricing behavior.

**Business Impact**

Keeping outliers ensures that high-revenue and niche routes are included in pricing analysis, supporting accurate revenue forecasting and strategic planning.

## 3. Univariate Analysis

Univariate analysis examines individual variables to understand their distribution, spread, and variability.
#### Why Histogram?

Histograms are effective for visualizing numerical distributions, identifying skewness, and understanding the impact of outliers.


In [ ]:
plt.figure()
plt.hist(airline['fare'], bins=50)
plt.title('Distribution of Air Fares')
plt.xlabel('Fare (USD)')
plt.ylabel('Frequency')
plt.show()


### Interpretation – Distribution of Air Fares

The fare distribution is right-skewed, with most tickets priced in the lower to mid range and a small number of very high fares. This suggests that premium or long-distance flights exist but are less common.

**Business impact:**

Helps the company identify typical pricing levels and investigate unusually high fares for potential pricing optimization.

In [ ]:

plt.figure()
plt.hist(airline['passengers'], bins=50)
plt.title('Distribution of Passengers')
plt.xlabel('Number of Passengers')
plt.ylabel('Frequency')
plt.show()


### Interpretation – Distribution of Passengers

The passenger distribution shows that most routes carry a moderate number of passengers, while a few routes handle exceptionally high volumes. This indicates uneven demand across routes.

**Business impact:**

Supports capacity planning by identifying which routes may require larger aircraft or increased frequency.

In [ ]:

route_passengers = airline.groupby(['airport_1','airport_2'])['passengers'].mean().sort_values(ascending=False).head(10)
plt.figure(figsize=(12,5))
route_passengers.plot(kind='bar', color='skyblue')
plt.title('1.4 Top 10 Routes by Passengers')
plt.ylabel('Passengers')
plt.xticks(rotation=45)
plt.show()


### Interpretation – Top 10 Routes by Passengers

The top routes carry significantly higher average passenger volumes compared to others, indicating consistently high demand between these airport pairs.

**Business impact:**

These routes are strong candidates for additional flights, higher-capacity aircraft, or priority resource allocation to maximize revenue and efficiency.

## 4. Bivariate Analysis

Bivariate analysis explores relationships between pairs of variables to identify associations.


**Why Distance vs Fare?**

Distance `nsmiles` is a core operational cost driver and is expected to influence airfare pricing.


In [ ]:
plt.figure()
plt.scatter(airline['nsmiles'], airline['fare'])
plt.title('Fare vs Distance')
plt.xlabel('Distance (Miles)')
plt.ylabel('Fare (USD)')
plt.show()


### Interpretation – Fare vs Distance

Fares generally increase as distance increases, though there is noticeable variation at similar distances. This suggests distance influences pricing but is not the only factor.

**Business impact:**

Helps validate distance-based pricing while highlighting routes where fares may be overpriced or underpriced relative to distance.

In [ ]:

plt.figure()
sns.scatterplot(
    data=airline,
    x='passengers',
    y='fare',
    alpha=0.6
)
plt.title('Passengers vs Fare')
plt.show()


### Interpretation – Passengers vs Fare

The scatter plot shows no strong linear relationship between passenger volume and fare. Routes with both high and low passenger counts can have similar fares.

**Business impact:**

Indicates that pricing is not driven by demand alone, suggesting opportunities to revisit pricing strategies on high-demand routes.

## 5. Multivariate Analysis

Multivariate analysis examines how multiple factors jointly influence airfare.


In [ ]:
import seaborn as sns
cols = ['fare', 'nsmiles', 'passengers', 'large_ms', 'lf_ms']
sns.pairplot(airline[cols], diag_kind='kde', plot_kws={'alpha':0.5, 's':20})
plt.suptitle('Multivariate Analysis: Pairwise Relationships', y=1.02)
plt.show()


### Interpretation – Multivariate Pairwise Relationships

The pairplot shows scatter plots for each variable pair and distribution plots on the diagonal.

Fare vs Distance (nsmiles) shows a positive trend: longer routes tend to cost more.

Fare vs Passengers shows weak correlation: high demand routes don’t always have higher fares.

Passengers vs Market Share (large_ms / lf_ms) shows little to moderate relationship: larger market share does not always imply more passengers.

The diagonal KDE plots reveal skewed distributions: fares are right-skewed, passengers mostly cluster at moderate values.

**Business Impact**

Identifies which factors (distance, market share, passenger count) are likely to influence pricing and demand.

Helps route planning, pricing strategy, and capacity allocation by revealing key patterns across multiple variables at once.

## 6. Correlation Analysis

Correlation analysis quantifies the strength and direction of relationships observed visually.


In [ ]:
# Correlation matrix for numeric variables
correlation = airline[cols].corr()
correlation

### Interpretation – Correlation Analysis

The correlation matrix shows the strength and direction of linear relationships between numeric variables:

Fare (fare) vs Distance (nsmiles): moderately positive (~0.5–0.6), meaning longer routes generally cost more.

Fare vs Passengers: very weak correlation, indicating demand does not strongly drive pricing.

Passengers vs Market Share (large_ms / lf_ms): low correlation, suggesting market share alone doesn’t explain passenger volumes.

Other pairs show minimal correlation, indicating variables largely vary independently.

**Business Impact**

Helps identify which factors have the most influence on pricing and passenger demand.

Guides route pricing, airline partnerships, and resource allocation based on objective numeric relationships.

## 7. Trend Detection

Trend analysis identifies long-term changes in airfare behavior.


### Why Group by Year?

Grouping by year reveals structural pricing trends and smooths short-term fluctuations.


In [ ]:
quarter_trend = airline.groupby(['year','quarter'])['passengers'].mean().reset_index()
airline['year'] = airline['year'].astype(int)

plt.figure(figsize=(12,6))
sns.lineplot(data=quarter_trend, x='year', y='passengers', hue='quarter', palette='tab10')
plt.title('Passenger Trend by Quarter Over Years')
plt.xlabel('Year')
plt.ylabel('Average Passengers')
plt.legend(title='Quarter')
plt.grid(True)
plt.show()

### Interpretation – Passenger Trend by Quarter Over Years

The lineplot shows average passengers per quarter for each year.

There are visible seasonal patterns: some quarters consistently have higher passenger numbers (peak travel seasons), while others are lower (off-peak).

Overall, the trend indicates growth or decline in passenger numbers over the years for each quarter.


**Business Impact:**

Helps the airport and airlines plan staffing, flight schedules, and resource allocation based on peak and off-peak periods.

Supports capacity planning and marketing campaigns for low-demand quarters.

In [ ]:
quarter_trend = airline.groupby(['year','quarter'])['fare'].mean().reset_index()
plt.figure(figsize=(12,6))
sns.lineplot(data=quarter_trend, x='year', y='fare', hue='quarter', palette='tab10')
plt.title('fare Trend by Quarter Over Years')
plt.xlabel('Year')
plt.ylabel('Average fare')
plt.legend(title='Quarter')
plt.grid(True)
plt.show()

Interpretation – Fare Trend by Quarter Over Years

The lineplot shows average fare per quarter over the years.

Fares fluctuate seasonally: some quarters have consistently higher fares (likely peak travel) and others lower.

Over time, there may be a general increase or decrease in fares, reflecting market or operational changes.


**Business Impact:**

Supports dynamic pricing strategies by identifying high-demand quarters where fares can be optimized.

Helps route revenue planning and forecasting for airlines.

In [ ]:

#  Create COVID period column 
airline['covid_period'] = airline['year'].apply(lambda x: 'Before COVID' if x < 2020 else 'During/Post COVID')

#  Calculate Revenue per flight 
airline['revenue'] = airline['passengers'] * airline['fare']

#Compare total and average revenue by COVID period
revenue_covid = airline.groupby('covid_period')['revenue'].agg(['sum','mean']).reset_index()
print(revenue_covid)

# Visualize Total Revenue
plt.figure(figsize=(6,5))
sns.barplot(data=revenue_covid, x='covid_period', y='sum')
plt.title('Total Revenue: Before vs During/Post COVID')
plt.ylabel('Total Revenue (USD)')
plt.xlabel('')
plt.show()

#Visualize Average Revenue per Flight
plt.figure(figsize=(6,5))
sns.barplot(data=revenue_covid, x='covid_period', y='mean')
plt.title('Average Revenue per Flight: Before vs During/Post COVID')
plt.ylabel('Average Revenue (USD)')
plt.xlabel('')
plt.show()


### Interpretation – COVID Impact on Revenue

Total Revenue:
The total revenue dropped significantly during/post COVID compared to before 2020. This reflects the overall decline in passenger numbers due to travel restrictions and reduced demand during the pandemic.
💡 Business Insight: Airlines need to recover lost traffic on high-demand routes and focus on marketing or promotions to bring passengers back.

Average Revenue per Flight:
The average revenue per flight may have remained stable or slightly changed. This indicates that pricing or revenue efficiency per flight was maintained, even though fewer flights were operating at full capacity.
💡 Business Insight: Airlines can maintain profitability by adjusting fares and optimizing capacity per flight rather than just increasing the number of flights.

## 8. Final Business Insight – Conclusion with Examples

The analysis shows that Q3 is the busiest travel period, with top routes and airports driving most passengers. Fares and revenue are closely tied to distance, but some high-demand routes are underpriced, revealing pricing opportunities. COVID caused a major drop in total revenue, though average revenue per flight remained stable, highlighting the need for recovery strategies.

**Recommendations with Examples:**

Focus on high-demand routes: Add extra flights for NY → LA and Chicago → Miami during Q3 to meet peak demand.

Adjust fares dynamically: Increase fares slightly on high-demand routes during summer, while offering discounts on underutilized flights in Q1/Q4.

Optimize staffing and gates: Allocate more check-in counters and staff at busiest airports like JFK and O’Hare during Q3.

Target off-peak periods: Promote weekend packages or discounted fares for spring and winter months to boost passenger numbers.

Monitor low-demand routes: Consider reducing frequency or temporarily suspending flights on routes with consistently low passengers, like smaller regional connections.



## 9. Conclusion

This EDA provides actionable insights for:

Pricing strategy

Route expansion

Market competition analysis

Demand forecasting